<a href="https://colab.research.google.com/github/niharika-sojitra/COMP-5800-Project/blob/main/Copy_of_T5_Fine_tuning_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://shivanandroy.com/fine-tune-t5-transformer-with-pytorch/

In [1]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.5 MB/s 
     |████████████████████████████████| 7.6 MB 20.3 MB/s 
     |████████████████████████████████| 182 kB 49.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 237 kB 5.1 MB/s 
     |████████████████████████████████| 51 kB 6.9 MB/s 
     |████████████████████████████████| 1.6 MB 51.3 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Project-5800/parallel_detoxification_dataset_small.tsv', sep='\t')
#df = pd.read_csv("https://github.com/s-nlp/parallel_detoxification_dataset/blob/main/parallel_detoxification_dataset_small.tsv", error_bad_lines=False)
#df = pd.read_csv('/content/drive/MyDrive/Project-5800/parallel.csv', error_bad_lines=False)
df.head()

,toxic_comment,civil_comment
0,. or the loud ass one - thousand ton beast roa...,or the loud one - thousand ton beast roaring ...
1,""" mandated "" and "" right fucking now "" would b...","""Mandated' and ""right now"" would be good."
2,""" mandated "" and "" right fucking now "" would b...","""mandated"" and"" right away"" would be good"
3,""" mandated "" and "" right fucking now "" would b...",mandated and right would be good
4,* neither * of my coworkers gave a shit when i...,Neither of my co-workers cared when it came to...


In [4]:
df.sample(10)

,toxic_comment,civil_comment
842,"holy fuck , who the hell thinks like that ?",Who can ever think this way ?
13,&gt i wouldn 't care how ignorant you are in y...,i wouldn 't care how ignorant you are in you w...
558,"food is really fucking cheap , jail time becau...","food is really cheap , prison time because peo..."
2123,then drench the fucking thing with bbq sauce !,Please drench the meat with bbq sauce
2174,they have been involved in so many fucked up t...,they have been involved in so many wrongs sinc...
1789,"silly you , its not about making america safe ...",I'ts not about making america safe or the reev...
1384,"kid , you should be watching the crap they cal...","Kid, you should be watching cartoons made for ..."
870,"holy shit , what is wrong with you ? .",what is wrong with you ?
982,i couldn t imagine fucking with an actual lion...,I couldnt imagine challenging a lion in its turf
1077,i mean it 's killed a shit ton of people and n...,This method will pay lot of precious lives and...


In [5]:
df["toxic_comment"] = "summarize: "+df["toxic_comment"]

In [6]:
df.head()

,toxic_comment,civil_comment
0,summarize: . or the loud ass one - thousand to...,or the loud one - thousand ton beast roaring ...
1,"summarize: "" mandated "" and "" right fucking no...","""Mandated' and ""right now"" would be good."
2,"summarize: "" mandated "" and "" right fucking no...","""mandated"" and"" right away"" would be good"
3,"summarize: "" mandated "" and "" right fucking no...",mandated and right would be good
4,summarize: * neither * of my coworkers gave a ...,Neither of my co-workers cared when it came to...


In [7]:
# Renaming the columns
#df.columns = ["input_text","target_text"]
# Adding a prefix. Here we shall keep "paraphrase" as a prefix.
#df["prefix"] = "paraphrase"

In [8]:
'''from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(df,test_size=0.1)'''

'from sklearn.model_selection import train_test_split\ntrain_data,test_data = train_test_split(df,test_size=0.1)'

In [9]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("toxic_comment", justify="center" ), Column("civil_comment", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [10]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [11]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, toxic_comment, civil_comment):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.civil_comment = self.data[civil_comment]
    self.toxic_comment = self.data[toxic_comment]

  def __len__(self):
    return len(self.civil_comment)

  def __getitem__(self, index):
    toxic_comment = str(self.toxic_comment[index])
    civil_comment = str(self.civil_comment[index])

    #cleaning data so as to ensure data is in string type
    toxic_comment = ' '.join(toxic_comment.split())
    civil_comment = ' '.join(civil_comment.split())

    source = self.tokenizer.batch_encode_plus([toxic_comment], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([civil_comment], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [12]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [13]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [14]:
def T5Trainer(dataframe, toxic_comment, civil_comment, model_params, output_dir="./outputs/" ):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[toxic_comment,civil_comment]]
  display_df(dataframe.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_toxic_comment_LENGTH"], model_params["MAX_civil_comment_LENGTH"], toxic_comment, civil_comment)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_toxic_comment_LENGTH"], model_params["MAX_civil_comment_LENGTH"], toxic_comment, civil_comment)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)
      
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [15]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":3,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_toxic_comment_LENGTH":512,  # max length of source text
    "MAX_civil_comment_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility 

}

In [ ]:
T5Trainer(dataframe=df[:500], toxic_comment ="toxic_comment", civil_comment="civil_comment", model_params=model_params, output_dir="outputs")

[15:46:56] [Model]: Loading t5-base...                                           <ipython-input-14-b0c32b477d75>:14
                                                                                                                   

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

...

FULL Dataset: (500, 2)

TRAIN Dataset: (400, 2)

TEST Dataset: (100, 2)

           [Initiating Fine Tuning]...                                           <ipython-input-14-b0c32b477d75>:74
                                                                                                                   

                      Training Status                       
+----------------------------------------------------------+
|Epoch | Steps |                    Loss                   |
|------+-------+-------------------------------------------|
|  0   |   0   | tensor(7.5278, grad_fn=<NllLossBackward0>)|
+----------------------------------------------------------+